In [22]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer
import re #import regular expression
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import string

In [2]:
file = open("tweet-data.txt", "r")
f = file.read()
print(f)

Siang, bisa langsung ditanyakan ke Kebidanan FK UB. Terima kasih. https://t.co/5P0KDHlK9r
Jadwal Kegiatan Dies Natalis ke-54 UB. Catat tanggalnya dan semarakkan acaranya. https://t.co/YZpBV1YFJK
Coming soon : Rangkaian Acara Dies Natalis ke-54 Universitas Brawijaya https://t.co/5xgu0Gj828
RT @UB_IO: Beasiswa AMINEF Untuk Pengajar Bahasa Inggris di UB. Deadline aplikasi tanggal 15 April 2017 https://t.co/Kg5Ked96bf @UB_Official
RT @UB_IO: Beasiswa untuk Training dan Studi Lanjut ke Belgia. @UB_Official @UBTV_Brawijaya https://t.co/eEHYiNzbrf
RT @UB_IT: Periksa Soal: cek jawaban kamu.Untuk kembali ke soal,klik nomornya.Tombol SELESAI di klik jika semua soal sudah dijawab &amp; nilaiâ€¦
RT @UB_IT: Tes kompetensi TI selama 90 menit/90 soal (30 Excel,30 P.Point,30 Word). Soal tiap orang berbeda krn bank soal lebih dari 3000.â€¦
RT @UnileverIDN: Prosesi penutupan Bulan Kesehatan Gigi Nasional 2016 di FKG Univ Brawijaya #BKGN #Pepsodent @UB_Official @TanyaPepsodent @â€¦
Selamat siang. Untuk j

In [3]:
#link normalize
def linkNormalize(self, text):
        text = re.sub(r"\s—\s", "", text)
        text = re.sub(r"http\S+", "", text)
        return text

In [4]:
text_linkNorm = linkNormalize('text',f)
text_linkNorm

"Siang, bisa langsung ditanyakan ke Kebidanan FK UB. Terima kasih. \nJadwal Kegiatan Dies Natalis ke-54 UB. Catat tanggalnya dan semarakkan acaranya. \nComing soon : Rangkaian Acara Dies Natalis ke-54 Universitas Brawijaya \nRT @UB_IO: Beasiswa AMINEF Untuk Pengajar Bahasa Inggris di UB. Deadline aplikasi tanggal 15 April 2017  @UB_Official\nRT @UB_IO: Beasiswa untuk Training dan Studi Lanjut ke Belgia. @UB_Official @UBTV_Brawijaya \nRT @UB_IT: Periksa Soal: cek jawaban kamu.Untuk kembali ke soal,klik nomornya.Tombol SELESAI di klik jika semua soal sudah dijawab &amp; nilaiâ€¦\nRT @UB_IT: Tes kompetensi TI selama 90 menit/90 soal (30 Excel,30 P.Point,30 Word). Soal tiap orang berbeda krn bank soal lebih dari 3000.â€¦\nRT @UnileverIDN: Prosesi penutupan Bulan Kesehatan Gigi Nasional 2016 di FKG Univ Brawijaya #BKGN #Pepsodent @UB_Official @TanyaPepsodent @â€¦\nSelamat siang. Untuk jadwal penerimaan mahasiswa S2 bisa dilihat di  atau diâ€¦ \nRT @kempanrb: bersama Korpri meneguhkan netral

In [5]:
#removal punctuation
text_removePunctuation = re.sub(r'[^a-zA-Z0-9]', ' ', text_linkNorm)
text_removePunctuation

'Siang  bisa langsung ditanyakan ke Kebidanan FK UB  Terima kasih   Jadwal Kegiatan Dies Natalis ke 54 UB  Catat tanggalnya dan semarakkan acaranya   Coming soon   Rangkaian Acara Dies Natalis ke 54 Universitas Brawijaya  RT  UB IO  Beasiswa AMINEF Untuk Pengajar Bahasa Inggris di UB  Deadline aplikasi tanggal 15 April 2017   UB Official RT  UB IO  Beasiswa untuk Training dan Studi Lanjut ke Belgia   UB Official  UBTV Brawijaya  RT  UB IT  Periksa Soal  cek jawaban kamu Untuk kembali ke soal klik nomornya Tombol SELESAI di klik jika semua soal sudah dijawab  amp  nilai    RT  UB IT  Tes kompetensi TI selama 90 menit 90 soal  30 Excel 30 P Point 30 Word   Soal tiap orang berbeda krn bank soal lebih dari 3000     RT  UnileverIDN  Prosesi penutupan Bulan Kesehatan Gigi Nasional 2016 di FKG Univ Brawijaya  BKGN  Pepsodent  UB Official  TanyaPepsodent      Selamat siang  Untuk jadwal penerimaan mahasiswa S2 bisa dilihat di  atau di     RT  kempanrb  bersama Korpri meneguhkan netralitas dan 

In [45]:
text_removeNumber = re.sub(r'[0-9]', ' ', text_removePunctuation)
text_removeNumber

'Siang  bisa langsung ditanyakan ke Kebidanan FK UB  Terima kasih   Jadwal Kegiatan Dies Natalis ke    UB  Catat tanggalnya dan semarakkan acaranya   Coming soon   Rangkaian Acara Dies Natalis ke    Universitas Brawijaya  RT  UB IO  Beasiswa AMINEF Untuk Pengajar Bahasa Inggris di UB  Deadline aplikasi tanggal    April        UB Official RT  UB IO  Beasiswa untuk Training dan Studi Lanjut ke Belgia   UB Official  UBTV Brawijaya  RT  UB IT  Periksa Soal  cek jawaban kamu Untuk kembali ke soal klik nomornya Tombol SELESAI di klik jika semua soal sudah dijawab  amp  nilai    RT  UB IT  Tes kompetensi TI selama    menit    soal     Excel    P Point    Word   Soal tiap orang berbeda krn bank soal lebih dari          RT  UnileverIDN  Prosesi penutupan Bulan Kesehatan Gigi Nasional      di FKG Univ Brawijaya  BKGN  Pepsodent  UB Official  TanyaPepsodent      Selamat siang  Untuk jadwal penerimaan mahasiswa S  bisa dilihat di  atau di     RT  kempanrb  bersama Korpri meneguhkan netralitas dan 

In [46]:
words = word_tokenize(text_removeNumber)
print(words)

['Siang', 'bisa', 'langsung', 'ditanyakan', 'ke', 'Kebidanan', 'FK', 'UB', 'Terima', 'kasih', 'Jadwal', 'Kegiatan', 'Dies', 'Natalis', 'ke', 'UB', 'Catat', 'tanggalnya', 'dan', 'semarakkan', 'acaranya', 'Coming', 'soon', 'Rangkaian', 'Acara', 'Dies', 'Natalis', 'ke', 'Universitas', 'Brawijaya', 'RT', 'UB', 'IO', 'Beasiswa', 'AMINEF', 'Untuk', 'Pengajar', 'Bahasa', 'Inggris', 'di', 'UB', 'Deadline', 'aplikasi', 'tanggal', 'April', 'UB', 'Official', 'RT', 'UB', 'IO', 'Beasiswa', 'untuk', 'Training', 'dan', 'Studi', 'Lanjut', 'ke', 'Belgia', 'UB', 'Official', 'UBTV', 'Brawijaya', 'RT', 'UB', 'IT', 'Periksa', 'Soal', 'cek', 'jawaban', 'kamu', 'Untuk', 'kembali', 'ke', 'soal', 'klik', 'nomornya', 'Tombol', 'SELESAI', 'di', 'klik', 'jika', 'semua', 'soal', 'sudah', 'dijawab', 'amp', 'nilai', 'RT', 'UB', 'IT', 'Tes', 'kompetensi', 'TI', 'selama', 'menit', 'soal', 'Excel', 'P', 'Point', 'Word', 'Soal', 'tiap', 'orang', 'berbeda', 'krn', 'bank', 'soal', 'lebih', 'dari', 'RT', 'UnileverIDN', 'Pr

In [55]:
#remove characters less than 4
text_new = re.sub(r'\b\w{1,3}\b', '', text_removeNumber)
text_new

'Siang  bisa langsung ditanyakan  Kebidanan    Terima kasih   Jadwal Kegiatan Dies Natalis       Catat tanggalnya  semarakkan acaranya   Coming soon   Rangkaian Acara Dies Natalis     Universitas Brawijaya       Beasiswa AMINEF Untuk Pengajar Bahasa Inggris    Deadline aplikasi tanggal    April         Official      Beasiswa untuk Training  Studi Lanjut  Belgia    Official  UBTV Brawijaya       Periksa Soal   jawaban kamu Untuk kembali  soal klik nomornya Tombol SELESAI  klik jika semua soal sudah dijawab    nilai          kompetensi  selama    menit    soal     Excel     Point    Word   Soal tiap orang berbeda  bank soal lebih dari            UnileverIDN  Prosesi penutupan Bulan Kesehatan Gigi Nasional        Univ Brawijaya  BKGN  Pepsodent   Official  TanyaPepsodent      Selamat siang  Untuk jadwal penerimaan mahasiswa   bisa dilihat   atau        kempanrb  bersama Korpri meneguhkan netralitas  meningkatkan profesionalisme Aparatur Sipil Negara  Selamat  KORPRI             Sosialisas

In [58]:
character = [' ']

In [59]:
def repeatcharNormalize(self, text):
        for i in range(len(character)):
            charac_long = 5
            while charac_long>=2:
                char = character[i]*charac_long 
                text = text.replace(char,character[i])
                charac_long -= 1
        return text

In [60]:
text_repeat = repeatcharNormalize('text',text_new)
text_repeat

'Siang bisa langsung ditanyakan Kebidanan Terima kasih Jadwal Kegiatan Dies Natalis Catat tanggalnya semarakkan acaranya Coming soon Rangkaian Acara Dies Natalis Universitas Brawijaya Beasiswa AMINEF Untuk Pengajar Bahasa Inggris Deadline aplikasi tanggal April Official Beasiswa untuk Training Studi Lanjut Belgia Official UBTV Brawijaya Periksa Soal jawaban kamu Untuk kembali soal klik nomornya Tombol SELESAI klik jika semua soal sudah dijawab nilai kompetensi selama menit soal Excel Point Word Soal tiap orang berbeda bank soal lebih dari UnileverIDN Prosesi penutupan Bulan Kesehatan Gigi Nasional Univ Brawijaya BKGN Pepsodent Official TanyaPepsodent Selamat siang Untuk jadwal penerimaan mahasiswa bisa dilihat atau kempanrb bersama Korpri meneguhkan netralitas meningkatkan profesionalisme Aparatur Sipil Negara Selamat KORPRI Sosialisasi Studi Beasiswa dari Swedia hari Rektorat Free entry kuota terbatas kursi Kesempatan terbaru untuk studi luar negeri dengan beasiswa Informasi lebih len

In [62]:
factory = StopWordRemoverFactory()
more_stopword = ['ini', 'saat','itu','adalah','sih', 'ucap']

data = factory.get_stop_words()+more_stopword
stopword = factory.create_stop_word_remover()
 
text_stopword = stopword.remove(f)
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '